## Setting (Initial Code)

In [ ]:
import os

def maybe_mkdir_p(directory: str) -> None:
    os.makedirs(directory, exist_ok=True)

# maic_dir = '/mnt/backup/'
# base_dir = os.path.join(maic_dir, 'working')
base_dir = os.getcwd()
input_dir = '/mnt/NM/dataset/'
temp_dir = os.path.join(base_dir, 'models/temp/convert_data/')

# maybe_mkdir_p(base_dir)
maybe_mkdir_p(temp_dir)


try:
    import nnunet
except:
    respository_dir = os.path.join(base_dir, 'models/module')
    os.chdir(respository_dir)
    ! pip install -e .
    os.chdir(base_dir)



## Setting ()

In [ ]:
# maic_dir = '/mnt/backup/'
# base_dir = os.path.join(maic_dir, 'working')
base_dir = os.getcwd()
input_dir = '/mnt/NM/dataset/'
temp_dir = os.path.join(base_dir, 'models/temp/convert_data/')

# maybe_mkdir_p(base_dir)
maybe_mkdir_p(temp_dir)

In [2]:
import shutil
import json
import numpy as np
import pandas as pd
from typing import List

from collections import OrderedDict
from scipy import special
import copy
import cv2
from tqdm import tqdm
import h5py

# must install
import pydicom
import nibabel as nib
import SimpleITK as sitk

In [ ]:
# Utils ..
def subdirs(folder: str, join: bool = True, prefix: str = None, suffix: str = None, sort: bool = True) -> List[str]:
    if join:
        l = os.path.join
    else:
        l = lambda x, y: y
    res = [l(folder, i) for i in os.listdir(folder) if os.path.isdir(os.path.join(folder, i))
           and (prefix is None or i.startswith(prefix))
           and (suffix is None or i.endswith(suffix))]
    if sort:
        res.sort()
    return res

def subfiles(folder: str, join: bool = True, prefix: str = None, suffix: str = None, sort: bool = True) -> List[str]:
    if join:
        l = os.path.join
    else:
        l = lambda x, y: y
    res = [l(folder, i) for i in os.listdir(folder) if os.path.isfile(os.path.join(folder, i))
           and (prefix is None or i.startswith(prefix))
           and (suffix is None or i.endswith(suffix))]
    if sort:
        res.sort()
    return res

In [6]:
task_name = 'Task55_PETCT'
# convert_name = 'Task555_PETCT'

main_dir = os.path.join(base_dir, 'models/main/nnUNet/nnunet')
mainT_dir = os.path.join(temp_dir, 'nnUNet/nnunet')

rawbase_dir = os.path.join(mainT_dir, 'nnUNet_raw_data_base/')

pp_dir = os.path.join(mainT_dir, 'preprocessed')
tasks_dir = os.path.join(mainT_dir, 'Tasks')
task_dir = os.path.join(tasks_dir, task_name)

model_dir = os.path.join(main_dir, 'nnUNet_trained_models')
# Prediction_dir = os.path.join(main_dir, 'nnUNet_Prediction_Results')
# result_dir = os.path.join(Prediction_dir, convert_name)
# staple_dir = os.path.join(Prediction_dir, 'staple')

# 1. Data preprocessing
maybe_mkdir_p(tasks_dir)
maybe_mkdir_p(temp_dir)

# 2. Directory
maybe_mkdir_p(main_dir)
maybe_mkdir_p(model_dir)
maybe_mkdir_p(pp_dir)

# 3. Directory
# maybe_mkdir_p(result_dir)
# maybe_mkdir_p(staple_dir)


#Environment Setting
os.environ['nnUNet_raw_data_base'] = rawbase_dir #os.path.join(mainT_dir, 'nnUNet_raw_data_base')
os.environ['nnUNet_preprocessed'] = pp_dir #os.path.join(mainT_dir, 'preprocessed')
os.environ['RESULTS_FOLDER'] = './models' #os.path.join(main_dir, 'nnUNet_trained_models')

print('Setting Completed!')

Setting Completed!


## Testdataset Preprocessing

In [ ]:
size_dic = {}
def hdf2nifti(hdf_folder: str, save_folder: str):
    # hdf_folder : [train_dir, test_dir] hdf5 file path
    # save_folder : [imagesTr, imagesTs] Save Folder path
    maybe_mkdir_p(os.path.join(save_folder, 'imagesTr'))
    maybe_mkdir_p(os.path.join(save_folder, 'imagesTs'))
    maybe_mkdir_p(os.path.join(save_folder, 'labelsTr'))
    print('Creating "{}" Image & Label ..'.format(os.path.basename(os.path.normpath(save_folder))))
    hdf5_files = os.listdir(hdf_folder)


    for hdf5_file in hdf5_files:


        hdf5_path = os.path.join(hdf_folder, hdf5_file)

        # image
        f_i = h5py.File(hdf5_path, 'r')
        ctarr = np.asarray(f_i['CT'])
        petarr = np.asarray(f_i['PET'])
        sizearr = np.asarray(f_i['Size'])
        f_i.close()
        
        size_dic[hdf5_file[:17]] = sizearr # new

        SLICE_SIZE_X, SLICE_SIZE_Y, SLICE_COUNT = ctarr.shape
        images = np.empty([SLICE_SIZE_X, SLICE_SIZE_Y, SLICE_COUNT, 0], dtype=np.single)

        image_ct = np.expand_dims(ctarr, axis=3)
        images = np.append(images, image_ct, axis=3)
        image_pet = np.expand_dims(petarr, axis=3)
        images = np.append(images, image_pet, axis=3)

        niim = nib.Nifti1Image(images, affine=np.eye(4))
        nib.save(niim, os.path.join(save_folder, 'imagesTs/{}.nii.gz'.format(hdf5_file[:17])))
        
        print('{} = sizearr : {}, ctarr : {}, petarr : {}'.format(hdf5_file, sizearr, ctarr.shape, petarr.shape))


    print('"{}" Image & Label Completed !!'.format(os.path.basename(os.path.normpath(save_folder))))
    print('Image Patient : {}'.format(len(os.listdir(input_dir))))

hdf2nifti(input_dir, task_dir)

In [ ]:
def json_mk(save_dir: str):
    # Path
    imagesTr = os.path.join(save_dir, 'imagesTr')
    imagesTs = os.path.join(save_dir, 'imagesTs')
    maybe_mkdir_p(imagesTr)
    maybe_mkdir_p(imagesTs)

    overwrite_json_file = True
    json_file_exist = False

    if os.path.exists(os.path.join(save_dir, 'dataset.json')):
        print('dataset.json already exist!')
        json_file_exist = True

    if json_file_exist == False or overwrite_json_file:

        json_dict = OrderedDict()
        json_dict['name'] = "PETCT"
        json_dict['description'] = "Medical Image AI Challenge 2021"
        json_dict['tensorImageSize'] = "4D"
        json_dict['reference'] = "https://maic.or.kr/competitions/"
        json_dict['licence'] = "SNUH"
        json_dict['release'] = "18/10/2021"

        json_dict['modality'] = {
            "0": "CT",
            "1": "PET"
        }
        json_dict['labels'] = {
            "0": "background",
            "1": "Aorta"
        }

        train_ids = sorted(os.listdir(imagesTr))
        test_ids = sorted(os.listdir(imagesTs))
        json_dict['numTraining'] = len(train_ids)
        json_dict['numTest'] = len(test_ids)

        json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

        json_dict['test'] = ["./imagesTs/%s" % i for i in test_ids] #(i[:i.find("_0000")])

        with open(os.path.join(save_dir, "dataset.json"), 'w') as f:
            json.dump(json_dict, f, indent=4, sort_keys=False)

        if os.path.exists(os.path.join(save_dir, 'dataset.json')):
            if json_file_exist == False:
                print('dataset.json created!')
            else:
                print('dataset.json overwritten!')
json_mk(task_dir)

In [ ]:

!nnUNet_convert_decathlon_task -i ./models/temp/convert_data/nnUNet/nnunet/Tasks/Task55_PETCT -output_task_id 555 # -i : task_dir
# !nnUNet_plan_and_preprocess -t 555 # --verify_dataset_integrity


## File path

## Prediction

In [13]:
!nnUNet_predict -i ./models/temp/convert_data/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task555_PETCT/imagesTs -o ./models/result -t 555 -tr nnUNetTrainerV2 -m 3d_fullres



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

using model stored in  ./model/nnUNet/2d/Task555_PETCT/nnUNetTrainerV2__nnUNetPlansv2.1
This model expects 2 input modalities for each image
Found 1 unique case ids, here are some examples: ['23010018_20141226']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
number of cases: 1
number of cases that still need to be predicted: 1
emptying cuda cache
loading parameters for folds, None
folds is None so we will automatically look for output folders (not using 'all'!)
found the following folds:  ['./model/nnUNet/2d/Task555_PETCT/nnUNetTrainerV2__nnUNetPlansv2.1/fold_0', './model/nn

## Submission

In [ ]:
def get_suv_params(ptarr, roi):
    roi = np.asarray(roi>0, dtype=np.float)
    suvmax = np.max(ptarr*roi)
    suvmean = np.sum(ptarr*roi)/np.sum(roi)
    return suvmax, suvmean

def get_vol_params(ptzoom, roi):
    roi = np.asarray(roi>0, dtype=np.float)
    return np.prod(ptzoom) * np.sum(roi)

In [ ]:
# PN_list = os.listdir('./models/result')
# PN_list.sort()

PN_list = subfiles('./models/result/', join=False, suffix='.nii.gz')

for pn_l in PN_list:
    _, ext = os.path.splitext(pn_l)
    if ext == '.gz':
        
        pn = pn_l[:-7]
        PATIENT_NUM = pn

        ptarr = np.array(nib.load('./models/temp/convert_data/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task555_PETCT/imagesTs/{}_0001.nii.gz'.format(pn)).dataobj)
        ctarr = np.array(nib.load('./models/temp/convert_data/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task555_PETCT/imagesTs/{}_0000.nii.gz'.format(pn)).dataobj)
        pred_arr = np.array(nib.load('./models/result/{}.nii.gz'.format(pn)).dataobj)

        # size = ([4.07283, 4.07283, 3.])
        size = size_dic[PATIENT_NUM]
        
        #Calculate Mean SUV and Max SUV

        suvmax, suvmean = get_suv_params(ptarr, pred_arr)

        #Calculate Volume

        aorvol = get_vol_params(size, pred_arr)
        
        PATIENT_NUM = pn

        data = {'case' : [PATIENT_NUM], 'PD_Aorta_volume' : [aorvol], 'PD_SUVmean' : [suvmean]}
        df = pd.DataFrame(data)

        # .to_csv 
        if not os.path.exists('submission.csv'):
            df.to_csv('submission.csv', index=False, mode='w')
        else:
            df.to_csv('submission.csv', index=False, mode='a', header=False)
